# Assignment 2

__Due date__: April 25, 2018 at 10 pm
__Submission__: IPython notebook to GauchoSpace

We have been building up to creating components for a dashboard app in Jupyter notebook:

- Data downloading function: `get_nba_data()` in a `03-Data-collection-and-manipulation.ipynb`. 
- Accessing pandas data frame and creating dictionary structures in `04-Pandas-Data-Frame.ipynb` and `05-Data-Frame-and-Visualization.ipynb`.
- Creating widgets for interactivity: `05-Data-Frame-and-Visualization.ipynb`.
- Plotting visualizations with Matplotlib and Seaborn: `05-Data-Frame-and-Visualization.ipynb`.

We can put these components together to create an interactive dashboard for creating something similar to the R package: https://github.com/toddwschneider/ballr

Your assignment is to create an interactive dashboard.

It doesn't have to be exactly the same as the package or what I have proposed. If you would like to create another visualization, that would be great as well. I will refer to __default option__ as continuing to do what we started in class: dashboard of shotchart similar to what [BallR package](https://github.com/toddwschneider/ballr) does. I will refer to __open-ended option__ as creating a dashboard of your choice.

Below, I specify some necessary components to your dashboard.

## Problem 1: Data Download

__Default option__: you can use the `get_nba_data()` function. No additional work is needed.

__Open-ended option__: you can choose to create a different dashboard. 

If you are familiar with http://stats.nba.com/, some stats pages will directly tell you how the data can be retrieved. For example, the data needed for [Tracking Shots Dashboard](http://stats.nba.com/player/201935/shots-dash/) comes from [this URL](http://stats.nba.com/stats/playerdashptshots?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PerMode=PerGame&Period=0&PlayerID=201935&Season=2017-18&SeasonSegment=&SeasonType=Playoffs&TeamID=0&VsConference=&VsDivision=). This link can be found under Tools > Developer tools menu (Control-Shift-I) if you are using [Google Chrome](https://www.google.com/chrome/). Once you open Developer tools, you need to reload the page (for F5). Then, go to `Networks` menu under Developer tools pane. Type into the `Filter` text input box, type `stats/`. This will find (if any) GET URL links that we can use. The URL corresponding to the [Tracking Shots Dashboard](http://stats.nba.com/player/201935/shots-dash/) is [here](http://stats.nba.com/stats/playerdashptshots?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PerMode=PerGame&Period=0&PlayerID=201935&Season=2017-18&SeasonSegment=&SeasonType=Playoffs&TeamID=0&VsConference=&VsDivision=). If you are feeling adventurous, you can use another data to create your dashboard with.

There are other interesting data sources: https://schoolofdata.org/2013/11/18/web-apis-for-non-programmers/ (note them some may be out of date since this is from 5 years ago!). If you would like to pursue a completely different data source, you are encouraged to. Keep in mind you want to create some dashboard to automatically update information. I can help you to determine wheter a site can be reverse engineered relatively easily if you choose to pursue this option.

Determine the set of parameters for creating an appropriate `param` dictionary. Test if your data download function works as intended.

In [13]:
# from Lecture Four:

import pandas as pd

def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

In [14]:
params = {'LeagueID':'00'}
teams = get_nba_data('commonTeamYears', params)

teams

,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION
0,00,1610612737,1949,2017,ATL
1,00,1610612738,1946,2017,BOS
2,00,1610612739,1970,2017,CLE
3,00,1610612740,2002,2017,NOP
4,00,1610612741,1966,2017,CHI
5,00,1610612742,1980,2017,DAL
6,00,1610612743,1976,2017,DEN
7,00,1610612744,1946,2017,GSW
8,00,1610612745,1967,2017,HOU
9,00,1610612746,1970,2017,LAC


## Problem 2: Creating interactive widgets

__Default option__: create at least one more widgets (three total) for specifying data downloads. For example, you can change year, opposing team, etc.

__Open-ended option__: create at least one widget for specifying data downloads.

In this problem, you will simply create widget(s). Next section will be showing how you can combine them.

## Problem 3: Downloading data with changing widget states

__Both options__: Add event handlers (`observe`, `on_click`, etc) to be called when some widget changes to another state. Make sure this works as expected.

## Problem 4: Data transformation and visualization

__Default option__: create at least two data transformation using split-apply-combine approach. Some ideas are, 

- What is the shooting average against different teams? You would split based on opposing team, compute the average, and plot a bar chart. 

- What is the shooting average over different periods per game? You would split based on periods and game, then plot the changing shooting average over periods. Is this helpful? Why? Why not?

- Any other setting in which you would need to split-apply-combine to calculate a summary statistic.

- A setting of your choosing

Plot your result

__Open-ended option__: create at least one data transformation using split-apply-combine approach.

Plot your result

## Notes

- Open-ended option will be a lot more work; however, you can use this towards building up your final projects if you so choose. 

- Label figures, and explain your steps. PSTAT 234 students' work are expected to be more refined.

- Exceptional assignments will receive extra credit.